# [Klasifikasi] Submission Akhir BMLP - Ridho Bintang Aulia

**Nama:** Ridho Bintang Aulia

**Dataset:** Hasil Clustering dari Bank Transaction Dataset for Fraud Detection (Modified)

---

## Penting

Notebook ini berisi proses **klasifikasi** menggunakan hasil clustering sebagai label target. Proyek ini mencakup:
1. Memuat Dataset Hasil Clustering
2. Data Splitting
3. Membangun Model Klasifikasi (Decision Tree)
4. Eksplorasi Algoritma Lain (Random Forest)
5. Hyperparameter Tuning

Setiap tahapan dilengkapi dengan penjelasan metode, alasan penggunaan, dan interpretasi hasil.

## 1. Import Library

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
import joblib
import warnings
warnings.filterwarnings('ignore')

print('Semua library berhasil diimpor!')

Semua library berhasil diimpor!


## 2. Memuat Dataset dari Hasil Clustering

Memuat dataset hasil clustering yang memiliki fitur Target.

**Metode:** Menggunakan `pd.read_csv()` untuk memuat file `data_clustering_inverse.csv` yang merupakan hasil dari notebook clustering.

**Alasan:** Dataset ini sudah memiliki kolom Target hasil clustering yang akan digunakan sebagai label untuk klasifikasi. Menggunakan data inverse agar fitur memiliki makna yang lebih jelas.

In [2]:
# Gunakan dataset hasil clustering yang memiliki fitur Target
df = pd.read_csv('data_clustering_inverse.csv')
print(f'Dataset berhasil dimuat dengan {df.shape[0]} baris dan {df.shape[1]} kolom.')
print(f'\nKolom: {df.columns.tolist()}')

Dataset berhasil dimuat dengan 1501 baris dan 12 kolom.

Kolom: ['TransactionAmount', 'TransactionType', 'Location', 'Channel', 'CustomerAge', 'CustomerOccupation', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'TransactionAmount_Bin', 'AccountBalance_Bin', 'Target']


In [3]:
# Tampilkan 5 baris pertama dengan function head
df.head()

,TransactionAmount,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,TransactionAmount_Bin,AccountBalance_Bin,Target
0,14.09,Debit,San Diego,ATM,70.0,Doctor,81.0,1.0,5112.21,Low,High,1
1,376.24,Debit,Houston,ATM,68.0,Doctor,141.0,1.0,13758.91,Very High,Very High,0
2,126.29,Debit,Mesa,Online,19.0,Student,56.0,1.0,1122.35,Medium,Low,3
3,184.50,Debit,Raleigh,Online,26.0,Student,25.0,1.0,8569.06,Medium,Very High,3
4,92.15,Debit,Oklahoma City,ATM,18.0,Student,172.0,1.0,781.68,Medium,Low,3


### (OPSIONAL) Feature Encoding

Karena menggunakan data inverse yang memiliki kolom kategorikal dalam bentuk string, kita perlu melakukan encoding ulang agar bisa digunakan oleh model klasifikasi.

**Metode:** `LabelEncoder()` untuk mengubah fitur kategorikal menjadi numerik.

**Alasan:** Algoritma Decision Tree dan Random Forest di scikit-learn memerlukan input numerik.

In [4]:
# Feature encoding untuk kolom kategorikal
from sklearn.preprocessing import LabelEncoder

categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f'Kolom kategorikal: {categorical_cols}')

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
    print(f'  {col}: encoded successfully')

print(f'\nDataset setelah encoding:')
df.head()

Kolom kategorikal: ['TransactionType', 'Location', 'Channel', 'CustomerOccupation', 'TransactionAmount_Bin', 'AccountBalance_Bin']


  TransactionType: encoded successfully
  Location: encoded successfully
  Channel: encoded successfully
  CustomerOccupation: encoded successfully
  TransactionAmount_Bin: encoded successfully
  AccountBalance_Bin: encoded successfully

Dataset setelah encoding:


,TransactionAmount,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,TransactionAmount_Bin,AccountBalance_Bin,Target
0,14.09,0,35,0,70.0,0,81.0,1.0,5112.21,1,0,1
1,376.24,0,14,0,68.0,0,141.0,1.0,13758.91,3,3,0
2,126.29,0,22,2,19.0,3,56.0,1.0,1122.35,2,1,3
3,184.50,0,32,2,26.0,3,25.0,1.0,8569.06,2,3,3
4,92.15,0,27,0,18.0,3,172.0,1.0,781.68,2,1,3


## 3. Data Splitting

**Metode:** `train_test_split()` dengan rasio 80:20 untuk membagi data menjadi training set dan testing set.

**Alasan:** Pembagian data memastikan bahwa model dilatih pada data training dan dievaluasi pada data testing yang belum pernah dilihat sebelumnya, sehingga mengukur kemampuan generalisasi model.

**Hasil yang didapat:** Dataset terbagi menjadi X_train, X_test, y_train, y_test.

In [5]:
# Menggunakan train_test_split() untuk melakukan pembagian dataset.
X = df.drop(columns=['Target'])
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f'Jumlah fitur: {X.shape[1]}')
print(f'\nUkuran data training: {X_train.shape[0]} ({X_train.shape[0]/len(X)*100:.1f}%)')
print(f'Ukuran data testing: {X_test.shape[0]} ({X_test.shape[0]/len(X)*100:.1f}%)')
print(f'\nDistribusi Target di training set:')
print(y_train.value_counts().sort_index())
print(f'\nDistribusi Target di testing set:')
print(y_test.value_counts().sort_index())

Jumlah fitur: 11

Ukuran data training: 1200 (79.9%)
Ukuran data testing: 301 (20.1%)

Distribusi Target di training set:
Target
0    278
1    448
2    230
3    244
Name: count, dtype: int64

Distribusi Target di testing set:
Target
0     70
1    112
2     58
3     61
Name: count, dtype: int64


## 4. Membangun Model Klasifikasi

### Decision Tree Classifier

**Metode:** `DecisionTreeClassifier()` dari scikit-learn.

**Alasan:** Decision Tree adalah model klasifikasi yang mudah diinterpretasi, mampu menangani data numerik dan kategorikal, serta tidak memerlukan feature scaling. Model ini menjadi baseline yang baik untuk klasifikasi.

**Hasil yang didapat:** Model Decision Tree yang dilatih pada data training.

In [6]:
# Buatlah model klasifikasi menggunakan Decision Tree
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Prediksi
y_pred_dt = dt_model.predict(X_test)

# Evaluasi
print('=== Evaluasi Decision Tree ===')
print(f'Accuracy : {accuracy_score(y_test, y_pred_dt):.4f}')
print(f'Precision: {precision_score(y_test, y_pred_dt, average="weighted"):.4f}')
print(f'Recall   : {recall_score(y_test, y_pred_dt, average="weighted"):.4f}')
print(f'F1-Score : {f1_score(y_test, y_pred_dt, average="weighted"):.4f}')
print(f'\nClassification Report:')
print(classification_report(y_test, y_pred_dt))

=== Evaluasi Decision Tree ===
Accuracy : 0.9900
Precision: 0.9901
Recall   : 0.9900
F1-Score : 0.9901

Classification Report:


              precision    recall  f1-score   support

           0       0.97      0.99      0.98        70
           1       1.00      1.00      1.00       112
           2       0.98      0.98      0.98        58
           3       1.00      0.98      0.99        61

    accuracy                           0.99       301
   macro avg       0.99      0.99      0.99       301
weighted avg       0.99      0.99      0.99       301



In [7]:
# Menyimpan Model
joblib.dump(dt_model, 'decision_tree_model.h5')
print('Model Decision Tree berhasil disimpan sebagai decision_tree_model.h5')

Model Decision Tree berhasil disimpan sebagai decision_tree_model.h5


## 5. Memenuhi Kriteria Skilled dan Advanced

### Penilaian (Opsional)

#### Skilled - Kriteria 5: Algoritma Klasifikasi Lain

**Metode:** `RandomForestClassifier()` sebagai algoritma klasifikasi tambahan.

**Alasan:** Random Forest adalah ensemble method yang menggabungkan multiple decision trees, sehingga cenderung memberikan performa yang lebih baik dan lebih tahan terhadap overfitting dibandingkan single Decision Tree.

**Hasil yang didapat:** Model Random Forest sebagai perbandingan dengan Decision Tree.

In [8]:
# Melatih model menggunakan algoritma klasifikasi scikit-learn selain Decision Tree.
# Contoh: RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_model.fit(X_train, y_train)

# Prediksi
y_pred_rf = rf_model.predict(X_test)

print('=== Evaluasi Random Forest ===')
print(f'Accuracy : {accuracy_score(y_test, y_pred_rf):.4f}')
print(f'Precision: {precision_score(y_test, y_pred_rf, average="weighted"):.4f}')
print(f'Recall   : {recall_score(y_test, y_pred_rf, average="weighted"):.4f}')
print(f'F1-Score : {f1_score(y_test, y_pred_rf, average="weighted"):.4f}')
print(f'\nClassification Report:')
print(classification_report(y_test, y_pred_rf))

=== Evaluasi Random Forest ===
Accuracy : 0.9967
Precision: 0.9967
Recall   : 0.9967
F1-Score : 0.9967

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        70
           1       1.00      1.00      1.00       112
           2       1.00      0.98      0.99        58
           3       1.00      1.00      1.00        61

    accuracy                           1.00       301
   macro avg       1.00      1.00      1.00       301
weighted avg       1.00      1.00      1.00       301



In [9]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score 
# pada seluruh algoritma yang sudah dibuat.
print('=' * 70)
print('PERBANDINGAN PERFORMA SELURUH MODEL KLASIFIKASI')
print('=' * 70)

results = {
    'Model': ['Decision Tree', 'Random Forest'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_dt),
        accuracy_score(y_test, y_pred_rf)
    ],
    'Precision': [
        precision_score(y_test, y_pred_dt, average='weighted'),
        precision_score(y_test, y_pred_rf, average='weighted')
    ],
    'Recall': [
        recall_score(y_test, y_pred_dt, average='weighted'),
        recall_score(y_test, y_pred_rf, average='weighted')
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_dt, average='weighted'),
        f1_score(y_test, y_pred_rf, average='weighted')
    ]
}

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

# Tentukan model terbaik
best_idx = results_df['F1-Score'].idxmax()
print(f'\nModel terbaik berdasarkan F1-Score: {results_df.loc[best_idx, "Model"]} '
      f'(F1-Score: {results_df.loc[best_idx, "F1-Score"]:.4f})')

PERBANDINGAN PERFORMA SELURUH MODEL KLASIFIKASI


        Model  Accuracy  Precision   Recall  F1-Score
Decision Tree  0.990033   0.990127 0.990033  0.990055
Random Forest  0.996678   0.996725 0.996678  0.996675

Model terbaik berdasarkan F1-Score: Random Forest (F1-Score: 0.9967)


In [10]:
# Menyimpan Model Selain Decision Tree
joblib.dump(rf_model, 'explore_RandomForest_classification.h5')
print('Model Random Forest berhasil disimpan sebagai explore_RandomForest_classification.h5')

Model Random Forest berhasil disimpan sebagai explore_RandomForest_classification.h5


### Penilaian (Opsional)

#### Advanced - Kriteria 5: Hyperparameter Tuning

**Metode:** `RandomizedSearchCV()` untuk melakukan hyperparameter tuning pada Random Forest Classifier.

**Alasan:** Hyperparameter tuning memungkinkan pencarian kombinasi parameter terbaik secara sistematis. RandomizedSearchCV lebih efisien dibanding GridSearchCV karena melakukan sampling acak dari parameter space, sehingga dapat menemukan parameter yang baik dalam waktu yang lebih singkat.

**Hasil yang didapat:** Model dengan parameter optimal yang memberikan performa terbaik.

In [11]:
# Lakukan Hyperparameter Tuning dan Latih ulang.
param_distributions = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

print('Memulai Hyperparameter Tuning dengan RandomizedSearchCV...')
print(f'Parameter space: {param_distributions}')

tuned_model = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=param_distributions,
    n_iter=30,
    cv=5,
    scoring='f1_weighted',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

tuned_model.fit(X_train, y_train)

print(f'\nParameter terbaik: {tuned_model.best_params_}')
print(f'Skor terbaik (CV): {tuned_model.best_score_:.4f}')

Memulai Hyperparameter Tuning dengan RandomizedSearchCV...
Parameter space: {'n_estimators': [50, 100, 200, 300], 'max_depth': [5, 10, 15, 20, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'max_features': ['sqrt', 'log2', None]}
Fitting 5 folds for each of 30 candidates, totalling 150 fits



Parameter terbaik: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 5}
Skor terbaik (CV): 0.9917


In [12]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score 
# pada algoritma yang sudah dituning.
best_model = tuned_model.best_estimator_
y_pred_tuned = best_model.predict(X_test)

print('=== Evaluasi Model Setelah Hyperparameter Tuning ===')
print(f'Accuracy : {accuracy_score(y_test, y_pred_tuned):.4f}')
print(f'Precision: {precision_score(y_test, y_pred_tuned, average="weighted"):.4f}')
print(f'Recall   : {recall_score(y_test, y_pred_tuned, average="weighted"):.4f}')
print(f'F1-Score : {f1_score(y_test, y_pred_tuned, average="weighted"):.4f}')
print(f'\nClassification Report:')
print(classification_report(y_test, y_pred_tuned))

# Perbandingan akhir
print('\n' + '=' * 70)
print('PERBANDINGAN AKHIR SELURUH MODEL')
print('=' * 70)

final_results = {
    'Model': ['Decision Tree', 'Random Forest', 'Random Forest (Tuned)'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_dt),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_tuned)
    ],
    'Precision': [
        precision_score(y_test, y_pred_dt, average='weighted'),
        precision_score(y_test, y_pred_rf, average='weighted'),
        precision_score(y_test, y_pred_tuned, average='weighted')
    ],
    'Recall': [
        recall_score(y_test, y_pred_dt, average='weighted'),
        recall_score(y_test, y_pred_rf, average='weighted'),
        recall_score(y_test, y_pred_tuned, average='weighted')
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_dt, average='weighted'),
        f1_score(y_test, y_pred_rf, average='weighted'),
        f1_score(y_test, y_pred_tuned, average='weighted')
    ]
}

final_df = pd.DataFrame(final_results)
print(final_df.to_string(index=False))

=== Evaluasi Model Setelah Hyperparameter Tuning ===
Accuracy : 0.9967
Precision: 0.9967
Recall   : 0.9967
F1-Score : 0.9967

Classification Report:


              precision    recall  f1-score   support

           0       0.99      1.00      0.99        70
           1       1.00      1.00      1.00       112
           2       1.00      0.98      0.99        58
           3       1.00      1.00      1.00        61

    accuracy                           1.00       301
   macro avg       1.00      1.00      1.00       301
weighted avg       1.00      1.00      1.00       301


PERBANDINGAN AKHIR SELURUH MODEL


                Model  Accuracy  Precision   Recall  F1-Score
        Decision Tree  0.990033   0.990127 0.990033  0.990055
        Random Forest  0.996678   0.996725 0.996678  0.996675
Random Forest (Tuned)  0.996678   0.996725 0.996678  0.996675


In [13]:
# Menyimpan Model hasil tuning
joblib.dump(best_model, 'tuning_classification.h5')
print('Model hasil tuning berhasil disimpan sebagai tuning_classification.h5')

# Simpan juga sebagai best_model_classification
joblib.dump(best_model, 'best_model_classification.h5')
print('Model terbaik juga disimpan sebagai best_model_classification.h5')

print('\n=== Semua model berhasil disimpan ===')
print('1. decision_tree_model.h5')
print('2. explore_RandomForest_classification.h5')
print('3. tuning_classification.h5')
print('4. best_model_classification.h5')

Model hasil tuning berhasil disimpan sebagai tuning_classification.h5


Model terbaik juga disimpan sebagai best_model_classification.h5

=== Semua model berhasil disimpan ===
1. decision_tree_model.h5
2. explore_RandomForest_classification.h5
3. tuning_classification.h5
4. best_model_classification.h5
